# Building Your First Campaign

This notebook will walk you through the steps to build a recommendation model for Allstore based on data collected from Amazon Outdoor equipment dataset. The goal is to recommend Amazon outdoor products that are relevant based on a particular user.

Note: You can bring you own dataset, but you need to ensure the schema of your dataset matches the intructions in this notebook.

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [4]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [5]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configure the data

Data is imported into Amazon Personalize through Amazon S3, below we will specify a bucket that you have created within AWS for the purposes of this exercise.

Below you will update the `bucket` variable to instead be set to the value that you created earlier in the CloudFormation steps, this should be in a text file from your earlier work. the `filename` does not need to be changed.

### Specify a Bucket and Data Output Location

In [6]:
bucket = "personalizedemoaseanunicorngymq419"       # replace with the name of your S3 bucket
filename = "allstore-ratings.csv"

#### Explore the Dataset

In [30]:
data = pd.read_csv('s3://personalizedemoaseanunicorngymq419/ratings.csv', names=['USER_ID', 'ITEM_ID', 'TIMESTAMP'], header=1)
pd.set_option('display.max_rows', 4)

data

,USER_ID,ITEM_ID,TIMESTAMP
0,A1L5P841VIO02V,1881509818,1328140800
1,AB2W04NI4OEAD,1881509818,1330387200
...,...,...,...
253014,APRNS6DB68LLV,B00LFPS0CY,1405900800
253015,A3UJRNI8UR4871,B00LFPS0CY,1405382400


## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [40]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

Here you can see below that we are picking the `aws-personalized-ranking` recipe.

In [105]:
recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking" 

In [ ]:
# aws-personalized-ranking selected for this solution. 
# Users are free to choose other recipes (such as SIMS, Personalize_Rakings and Popularity_count)

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [106]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-personal-ranking",   # Please change the solution name if you are changing the recipe
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:387269085412:solution/personalize-demo-soln-personal-ranking",
  "ResponseMetadata": {
    "RequestId": "2b1c73c7-50c8-4ab0-9003-530e2ebb30e2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 17 Dec 2019 06:40:06 GMT",
      "x-amzn-requestid": "2b1c73c7-50c8-4ab0-9003-530e2ebb30e2",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [107]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:387269085412:solution/personalize-demo-soln-personal-ranking/23e031fd",
  "ResponseMetadata": {
    "RequestId": "20020eb3-9d89-4a29-99df-1ce4b07c6e21",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 17 Dec 2019 06:40:21 GMT",
      "x-amzn-requestid": "20020eb3-9d89-4a29-99df-1ce4b07c6e21",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [45]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:387269085412:solution/personalize-demo-soln-hrnn/acd15615",
  "metrics": {
    "coverage": 0.0595,
    "mean_reciprocal_rank_at_25": 0.0084,
    "normalized_discounted_cumulative_gain_at_10": 0.0115,
    "normalized_discounted_cumulative_gain_at_25": 0.0146,
    "normalized_discounted_cumulative_gain_at_5": 0.01,
    "precision_at_10": 0.0017,
    "precision_at_25": 0.0012,
    "precision_at_5": 0.0025
  },
  "ResponseMetadata": {
    "RequestId": "d15db9bc-0b9d-4b5d-8fc5-3daa2a13ae74",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 17 Dec 2019 03:53:47 GMT",
      "x-amzn-requestid": "d15db9bc-0b9d-4b5d-8fc5-3daa2a13ae74",
      "content-length": "408",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [46]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ranking-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:387269085412:campaign/personalize-demo-camp",
  "ResponseMetadata": {
    "RequestId": "0abff5cb-a59e-4490-8712-fd6410da1a0b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 17 Dec 2019 03:54:07 GMT",
      "x-amzn-requestid": "0abff5cb-a59e-4490-8712-fd6410da1a0b",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [47]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting the Allstore merchandise (item descriptions) information to show for recommendations instead of just IDs.

In [89]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

USER: A10CKY22VNWXHK


In [90]:
# First load items into memory
items = pd.read_csv('s3://personalizedemoaseanunicorngymq419/convertcsv.csv', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_allstore_products(product_id):
    """
    Takes in an ID, returns a title
    """
    product_id = str(product_id)
    return items.loc[product_id]['TITLE']


#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [91]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

item_list


recommendation_list = []

for item in item_list:
    title = get_allstore_products(item['itemId'])
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

Recommendations for user:  A10CKY22VNWXHK


,Item Description
0,Rothco 550lb. Type III Nylon Paracord
1,Ultralight Backpacking Canister Camp Stove wit...
2,Emergency Mylar Thermal Blankets (Pack of 10)
3,Mirrycle MTB Bar End Mountain Bicycle Mirror
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot
5,Serfas TCPG Bicycle Floor Pump
6,Light My Fire Titanium Spork
7,Outdoor Products 3-Pack Ultimate Dry Sack
8,LifeStraw Personal Water Filter
9,MetroFlash Safety Zone Tail Light


## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `2.View_Campaign_And_Interactions.ipynb`



## Notes for the Next Notebook:

There are a few values you will need for the next notebook, execute the cells below to store them so they can be copied and pasted into the next part of the exercise.

In [92]:
%store campaign_arn

Stored 'campaign_arn' (str)


In [93]:
%store dataset_group_arn

Stored 'dataset_group_arn' (str)


In [94]:
%store solution_version_arn

Stored 'solution_version_arn' (str)


In [95]:
%store solution_arn

Stored 'solution_arn' (str)


In [96]:
%store dataset_arn

Stored 'dataset_arn' (str)


In [97]:
%store campaign_arn

Stored 'campaign_arn' (str)


In [98]:
%store schema_arn

Stored 'schema_arn' (str)


In [99]:
%store bucket

Stored 'bucket' (str)


In [100]:
%store filename

Stored 'filename' (str)


In [101]:
%store role_name

Stored 'role_name' (str)


In [102]:
%store recommendations_df

Stored 'recommendations_df' (DataFrame)


In [103]:
%store user_id

Stored 'user_id' (str)
